# INTRODUCCIÓN

La pandemia trajo muchos cambios en la rutina de la gente, entre ellos el aumento considerable de personas que prefieren quedarse en casa leyendo libros. Esto hizo que las startups se enfocaran en desarrollar nuevas aplicaciones. 

El objetivo de esta tarea es consultar la base de datos para generar una propuesta de valor para un nuevo producto.

# TABLA DE CONTENIDO

# 1. Estudiar las tablas
# 2. Consultas y resultados
# 3. Conclusiones

In [1]:
# importar librerías
import pandas as pd
from sqlalchemy import create_engine


db_config = {'user': 'practicum_student',         # nombre de usuario
             'pwd': 's65BlTKV3faNIGhmvJVzOqhs', # contraseña
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432,              # puerto de conexión
             'db': 'data-analyst-final-project-db'}          # nombre de la base de datos

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                                     db_config['pwd'],
                                                                       db_config['host'],
                                                                       db_config['port'],
                                                                       db_config['db'])

engine = create_engine(connection_string, connect_args={'sslmode':'require'})

In [2]:
# para ejecutar consultas
# pd.io.sql.read_sql(query, con = engine)

# ESTUDIAR LAS TABLAS

In [3]:
#mostrando las primeras filas de la tabla "books"
query = "SELECT * FROM books LIMIT 5"
pd.io.sql.read_sql(query, con = engine)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [4]:
#mostrando las primeras filas de la tabla "authors"
query = "SELECT * FROM authors LIMIT 5"
pd.io.sql.read_sql(query, con = engine)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [5]:
#mostrando las primeras filas de la tabla "publishers"
query = "SELECT * FROM publishers LIMIT 5"
pd.io.sql.read_sql(query, con = engine)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [6]:
#mostrando las primeras filas de la tabla "ratings"
query = "SELECT * FROM ratings LIMIT 5"
pd.io.sql.read_sql(query, con = engine)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [7]:
#mostrando las primeras filas de la tabla "reviews"
query = "SELECT * FROM reviews LIMIT 5"
pd.io.sql.read_sql(query, con = engine)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


# CONSULTAS Y RESULTADOS

In [8]:
#numero de libros publicados después del 1 de enero del 2000
query1 = """SELECT COUNT(book_id) 
            FROM books 
            WHERE (publication_date :: date > '2000-01-01')"""
pd.io.sql.read_sql(query1, con = engine)

,count
0,819


In [9]:
#numero de reseñas de usuario y la calificación promedio para cada libro
query2 = """SELECT books.title AS title, rating, review 
            FROM books 
            INNER JOIN 
                (SELECT book_id, AVG(rating) AS rating 
                FROM ratings 
                GROUP BY book_id) AS avg_ratings 
                    ON books.book_id = avg_ratings.book_id 
                    INNER JOIN 
                        (SELECT book_id, COUNT(review_id) AS review 
                        FROM reviews 
                        GROUP BY book_id) AS count_reviews 
                            ON books.book_id = count_reviews.book_id 
                            ORDER BY review DESC"""
pd.io.sql.read_sql(query2, con = engine)

,title,rating,review
0,Twilight (Twilight #1),3.662500,7
1,The Road,3.772727,6
2,The Book Thief,4.264151,6
3,The Glass Castle,4.206897,6
4,Water for Elephants,3.977273,6
...,...,...,...
989,Naked Empire (Sword of Truth #8),3.500000,1
990,Moo Baa La La La!,3.000000,1
991,Merrick (The Vampire Chronicles #7),4.000000,1
992,Babyville,3.500000,1


In [10]:
#editorial con mayor numero de libros con más de 50 páginas
query3 = """SELECT publishers.publisher, publishers.publisher_id, books_count 
            FROM publishers 
            INNER JOIN 
                (SELECT publisher_id, COUNT(book_id) AS books_count 
                FROM books WHERE (num_pages > 50) 
                GROUP BY publisher_id 
                ORDER BY books_count DESC) AS count_books 
                    ON publishers.publisher_id = count_books.publisher_id 
                    ORDER BY books_count DESC LIMIT 1"""
pd.io.sql.read_sql(query3, con = engine)

,publisher,publisher_id,books_count
0,Penguin Books,212,42


In [11]:
#autor con mayor calificación promedio de libro (el libro debe tener al menos 50 calificaciones)
query4 = """SELECT author 
            FROM authors 
            WHERE author_id IN 
                (SELECT author_id 
                FROM books 
                WHERE book_id IN 
                    (SELECT book_id 
                    FROM ratings 
                    GROUP BY book_id 
                    HAVING COUNT(rating) >= 50 
                    ORDER BY AVG(rating) 
                    DESC LIMIT 1))"""
pd.io.sql.read_sql(query4, con = engine)

,author
0,J.K. Rowling/Mary GrandPré


In [12]:
query5 = """SELECT COUNT(review_id) / COUNT(DISTINCT username) AS media 
            FROM reviews 
            WHERE username IN 
                (SELECT username 
                FROM ratings 
                GROUP BY username 
                HAVING COUNT(rating_id) > 50)"""
pd.io.sql.read_sql(query5, con = engine)

,media
0,24


In [13]:
query5 = """SELECT AVG(count_reviews) as avg_reviews
            FROM
                (SELECT username, COUNT(review_id) as count_reviews
                FROM reviews
                WHERE username IN
                    (SELECT username 
                    FROM ratings 
                    GROUP BY username 
                    HAVING COUNT(rating_id) > 50)
                GROUP BY username) AS hola"""
pd.io.sql.read_sql(query5, con = engine)

,avg_reviews
0,24.333333


# CONCLUSIONES

- De un total de 1000 libros, 819 se publicaron después del 1 de enero del 2020, es decir, la base de datos contiene en su mmayoría libros que se publicaron durante o después de la pandemia. 

- La calificación promedio varía entre 1.5 a 5 puntos y el número de reseñas de cada libro varía de 1 a 7. El rating y el número de reseñas varía bastante, es decir, la base de datos contiene una gran variabilidad. La tabla se muestra a continuación:

In [14]:
pd.io.sql.read_sql(query2, con = engine)

,title,rating,review
0,Twilight (Twilight #1),3.662500,7
1,The Road,3.772727,6
2,The Book Thief,4.264151,6
3,The Glass Castle,4.206897,6
4,Water for Elephants,3.977273,6
...,...,...,...
989,Naked Empire (Sword of Truth #8),3.500000,1
990,Moo Baa La La La!,3.000000,1
991,Merrick (The Vampire Chronicles #7),4.000000,1
992,Babyville,3.500000,1


- La editorial con mayor número de libros (con más de 50 páginas) es Penguin Books con 42 libros. Por lo que se recomienda buscar libros de esta editorial para las nuevas aplicaciones.

- El autor con mejor calificación promedio del libro (libros con al menos 50 calificaciones) es J.K. Rowling/Mary GrandPré, sin duda una autor que no puede faltar en ninguna aplicación.

- Los usuarios que calificaron sobre 50 libros han realizado en promedio 24,33 reseñas. Los usuarios que interactuan bastante con la aplicación tienden a realizar también bastantes reseñas. Esto es importante para analizar que tipo de libros subir o bajar de la app.